In [1]:
import seaborn as sns
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
from tqdm import tqdm
import pickle
import seaborn as sns
import multiprocessing
import re
from importlib import reload

In [2]:
import sys
sys.path.insert(1, '/project/cper_neon_aop/cper_hls_veg_models/biomass/params')
sys.path.insert(1, '/project/cper_neon_aop/hls_nrt/fit')
sys.path.insert(1, '/project/cper_neon_aop/hls_nrt/extract')

In [56]:
reload(sys.modules["cper_bm_params_fit_kmeans_transect"])
#from cper_bm_params_fit_plot_transect import *
#from cper_bm_params_fit_spatial_transect import *
#from cper_bm_params_fit_ecosite_transect import *
#from cper_bm_params_fit_year_transect import *
from cper_bm_params_fit_kmeans_transect import *

In [44]:
#reload(sys.modules["hpc_setup"])
from hpc_setup import launch_dask
from ml_setup import fit_ols, fit_dnn

In [47]:
#client.cluster.close()
#client.close()

In [48]:
if backend == 'dask':
    from hpc_setup import launch_dask
    import psutil
    interface_prefs = [
        'ibp175s0',
        'ibp59s0',
        'enp24s0f0',
        'ens7f0',
        'ibp24s0',
        'ib0'
    ]
    interface = [x for x in interface_prefs if x in list(psutil.net_if_addrs().keys())][0]
    if len(interface) == 0:
        print('ERROR: Preferred interfaces not found on node!')
    else:
        print(interface)
    num_jobs=32
    client = launch_dask(cluster_loc=cluster_loc,
                         num_jobs=num_jobs,
                         mem_gb_per=4.0,
                         partition='atlas', 
                         duration='04:00:00',
                         slurm_opts={'interface': interface},
                          extra_directives=["--account=cper_neon_aop"],
                        wait_timeout=300,
                        debug=False)
    display(client)
    nthreads = len(client.nthreads())
else:
    nthreads = multiprocessing.cpu_count()

ib0
   setting up cluster on HPC...


<Client: 'tcp://172.21.15.148:39207' processes=0 threads=0, memory=0 B>

In [49]:
import torch
if torch.has_cuda and use_cuda:
    device='cuda'
    print(torch.cuda.device_count())
else:
    device='cpu'
print(device)

cpu


In [50]:
#reload(sys.modules["ml_setup"])
from ml_setup import make_model_dictionary
from ml_setup import run_ml_models

In [57]:
mod_dict = make_model_dictionary(var_names, y_col, device)

In [58]:
df = load_df(inPATH, ['Date', 'Date_mean'])

In [59]:
#df.groupby('ecosite').apply(lambda x: x['Pasture'].unique())

In [60]:
len(df['Id'].unique())

528

In [61]:
len(df)

9647

In [62]:
df.groupby('Kmeans').value_counts(['Season'])

Kmeans  Season 
0       June       1204
1       October     704
        June        336
2       October    1448
        June          8
3       October     796
        June         24
4       June       1161
5       October     844
6       June         99
        October      88
7       June        476
        October     412
8       June       1288
        October      80
9       October     443
        June        236
Name: count, dtype: int64

In [63]:
mod_dict['DNN']['fit'] = False
if prefix == 'transect':
    mod_dict['DNN']['param_grid']['batch_size'] =  [256]

In [64]:
if 'BAI_236' not in var_names or 'NDII7' not in var_names or 'NIR1' not in var_names:
    mod_dict['CPER_2022']['fit'] = False
    mod_dict['OLS_2022']['fit'] = False

In [65]:
#for k in mod_dict.keys():
#    if k not in ['SVR', 'HGBR']:
#        mod_dict[k]['fit'] = False

In [66]:
mod_dict['GBR']['fit'] = False
mod_dict['HGBR']['variable_importance'] = True

In [67]:
[k + str(mod_dict[k]['fit']) for k in mod_dict]

['CPER_2022True',
 'OLS_2022True',
 'OLSTrue',
 'LASSOTrue',
 'PLSTrue',
 'PCRTrue',
 'SVRTrue',
 'RFTrue',
 'GBRFalse',
 'HGBRTrue',
 'MLPFalse',
 'DNNFalse']

In [68]:
#client.restart()
#import distributed
#client = [x for x in distributed.client._global_clients.values()][0]
#display(client)
#nthreads = len(client.nthreads())

In [69]:
run_ml_models(nickname,
              mod_dict,
              df, 
              y_col,
              date_col,
              var_names,
              kfold_group,
              tuneby_group, 
              kfold_type,
              tune_kfold_type,
              outFILE_tmp, 
              outDIR,
              backend,
              nthreads,
              cper_mod_xfrm,
              cper_mod_xfrm_func,
             client)

0
....fitting CPER_2022 (time to fit: 0.02 secs)
....fitting OLS_2022 (time to fit: 0.03 secs)
....fitting OLS 

/project/cper_neon_aop/hls_nrt/fit/ml_setup.py:585: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  mod_dict[k]['formula_df'] = pd.concat([mod_dict[k]['formula_df'], pd.concat(df_results[0])])


(time to fit: 35.54 secs)
....fitting LASSO 

/project/cper_neon_aop/conda_envs/py_ml_env/lib/python3.9/site-packages/distributed/client.py:3157: UserWarning: Sending large graph of size 26.97 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
/project/cper_neon_aop/conda_envs/py_ml_env/lib/python3.9/site-packages/distributed/client.py:3157: UserWarning: Sending large graph of size 26.35 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
/project/cper_neon_aop/conda_envs/py_ml_env/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.949e+03, tolerance: 7.306e+01
  model = cd_fast.enet_coordinate_descent(
/project/cper_neon_aop/hls_nrt/fit/ml_setup.py:671: FutureWarning: The behavior of DataFrame concatenatio

(time to fit: 70.48 secs)
....fitting PLS 

...

(time to fit: 15.95 secs)
....fitting PLS (time to fit: 0.64 secs)
....fitting PCR (time to fit: 0.75 secs)
....fitting SVR (time to fit: 2.82 secs)
....fitting RF 

/project/cper_neon_aop/conda_envs/py_ml_env/lib/python3.9/site-packages/distributed/client.py:3157: UserWarning: Sending large graph of size 17.77 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


(time to fit: 50.97 secs)
Skipping GBR, params not set to fit.
....fitting HGBR (time to fit: 8.79 secs)
Skipping MLP, params not set to fit.
Skipping DNN, params not set to fit.
9
....fitting CPER_2022 (time to fit: 0.02 secs)
....fitting OLS_2022 (time to fit: 0.04 secs)
....fitting OLS (time to fit: 14.95 secs)
....fitting LASSO 

/project/cper_neon_aop/conda_envs/py_ml_env/lib/python3.9/site-packages/distributed/client.py:3157: UserWarning: Sending large graph of size 28.66 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
/project/cper_neon_aop/conda_envs/py_ml_env/lib/python3.9/site-packages/distributed/client.py:3157: UserWarning: Sending large graph of size 27.99 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
/project/cper_neon_aop/conda_envs/py_ml_env/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.387e+04, tolerance: 7.208e+01
  model = cd_fast.enet_coordinate_descent(


(time to fit: 17.81 secs)
....fitting PLS (time to fit: 0.59 secs)
....fitting PCR (time to fit: 0.8 secs)
....fitting SVR (time to fit: 3.56 secs)
....fitting RF (time to fit: 54.05 secs)
Skipping GBR, params not set to fit.
....fitting HGBR (time to fit: 8.33 secs)
Skipping MLP, params not set to fit.
Skipping DNN, params not set to fit.
